# Introducción

En el campo del NLP, existen multitud de causas por las cuales pueden aparecer errores en campos de texto. Algunas muy comunes pueden ser:
- Typos al escribir
- Errores de reconocimiento de un ASR
- Errores al transcribir imágenes a texto (OCR)

Aunque hoy en día existen modelos complejos basados en Deep Learning (como pueden ser los seq2seq), existen otros mucho menos complejos basados en determinar la similitud (o diferencia) entre strings en cuanto al número de cambios lexico-gráficos necesarios para transformar un string en otro.

# Distancia de Edición

La distancia de edición es un método para calcular la similitud entre cadenas de caracteres. Aunque veremos más ejemplos en el notebook de **Matching** y sucesivos, se introduce aquí para presentar un algoritmo de corrección ortográfica.

Dicha distancia tiene en cuenta el **número mínimo de operaciones** requeridas para transformar una cadena de caracteres en otra. También se utiliza en bioinformática para cuantificar la similitud entre cadenas de ADN.

En función de las operaciones que se permitan, existen **distintos tipos de Distancia de Edición**:

|                Distancia               	| Inserción 	| Eliminación 	| Sustitución 	| Transposición 	|
|:--------------------------------------:	|:---------:	|:-----------:	|:-----------:	|:-------------:	|
|               Levenshtein              	|     Y     	|      Y      	|      Y      	|       -       	|
| Subsecuencia Común <br>más Larga (LCS) 	|     Y     	|      Y      	|      -      	|       -       	|
|                Hamming \*               	|     -     	|      -      	|      Y      	|       -       	|
|         Damerau-Levenshtein \**         	|     Y     	|      Y      	|      -      	|       Y       	|
|                  Jaro                  	|     -     	|      -      	|      -      	|       Y       	|

- \* _Solo sirve para strings de igual longitud_
- \** _Transposición de dos caracteres adyacentes_

Además del número de operaciones, también debe definirse la penalización por cada operación. Una posibilidad es la de que cada operación suponga sumar +1 a la distancia entre dos strings.

> d('_cena_', '_coma_') = 2

Si se define un coste positivo se cumple lo siguiente:
- d(a, b) = 0, solo si a=b
- d(a, b) > 0, si a$\neq$b
- d(a, b) = d(b, a)

Hay que tener en cuenta que una edición no tiene porqué resultar en una palabra real.

## Distancia de Levenshtein

Operaciones permitidas: inserción, eliminación y sustitución.

<img src=https://wikimedia.org/api/rest_v1/media/math/render/svg/4520f5376b54613a5b0e6c6db46083989f901821>

En este artículo lo explican de maravilla: [link](https://medium.com/@ethannam/understanding-the-levenshtein-distance-equation-for-beginners-c4285a5604f0)

<img src=https://miro.medium.com/max/1108/1*bEWdxv_FoTQurG9fyS3nSA.jpeg width=300px>

## Utilidad

Encontrar palabras _cercanas_ entre sí en cuanto a su similitud lexico-gráfica permite, entre otras cosas, corregir pequeños errores o incorrecciones en un texto.

Veremos un algoritmo sencillo de _Spelling Correction_ desarrollado por  Peter Norvig. [Link](https://norvig.com/spell-correct.html).

# Lectura de datos

In [ ]:
import sys
sys.path.append('../..')

from utils import load_cinema_reviews

In [ ]:
# Path al directorio donde tenemos los datasets con las reviews
datasets_path = '../../datasets'
corpus_cine_folder = 'corpusCine'

In [ ]:
reviews_dict = load_cinema_reviews(datasets_path, corpus_cine_folder)

In [ ]:
reviews_list = list()

for key, item in reviews_dict.items():
    reviews_list.append(item.get('review_text'))

In [ ]:
reviews_list[0]

# Algoritmo de Peter Norvig para Spell Checking

Peter Norvig, ex-director de Calidad de búsqueda en Google, escribió este _simple_, pero eficiente, algoritmo para corregir errores ortográficos. Según comenta lo escribió en un vuelo transcontinental para explicar el funcionamiento de un corrector ortográfico _funcional_ de una manera _sencilla_

[Link](https://norvig.com/spell-correct.html)

<img src=https://upload.wikimedia.org/wikipedia/commons/b/b8/Peter_Norvig_in_2019.jpg width=300px>

## ¿Cómo funciona?

El objetivo principal es, dada una palabra _w_, un conjunto de candidatos que pudieran ser susceptibles de ser su corrección. A priori no podemos conocer si la palabra _w_ es correcta o incorrecta. Tampoco, de ser incorrecta, conocemos la palabra candidata (de existir) a la que debería ser corregida.

De esta manera, se buscará encontrar el candidato correcto (corrección) _c_ que, de todos los posibles candidatos, maximize la probabilidad de que, dada la palabra _w_, sea _c_ la corrección.

Esto es:
<center>argmax<sub>c $\in$ candidates </sub> P(c|w)</center>

Que por Bayes es equivalente a:
<center>argmax<sub>c $\in$ candidates </sub> P(c)P(w|c)/P(w)</center>

Donde P(w) puede eliminarse al ser la misma para cada posible candidato:
<center>argmax<sub>c $\in$ candidates </sub> P(c)P(w|c)</center>

Los partes de esta expresión son, por tanto:
1. Mecanismo de selección: _argmax_
Se elige el candidato que maximiza la probabilidad.
2. Modelo de candidatos: _c $\in$ candidates_
Elige los candidatos a tener en cuenta.
3. Modelo de lenguaje: _P(c)_
La probabilidad de que _c_ apareza como palabra en un texto (probabilidad de ocurrencia)
4. Modelo de error: _P(w|c)_
La probabilidad de que w fuese escrita cuando realmente se quería escribir c.

In [11]:
import re
from collections import Counter

In [12]:
def words(text): return re.findall(r'\w+', text.lower())

In [13]:
text = ' '.join(list(filter(None, reviews_list)))
WORDS = Counter(words(text))

In [14]:
WORDS.most_common(100)

[('de', 110320),
 ('que', 75818),
 ('la', 69958),
 ('y', 54669),
 ('en', 49071),
 ('el', 47192),
 ('a', 38985),
 ('un', 30316),
 ('una', 24828),
 ('es', 24347),
 ('los', 23787),
 ('no', 22500),
 ('se', 20925),
 ('con', 20596),
 ('por', 18558),
 ('del', 17972),
 ('su', 16074),
 ('lo', 14570),
 ('las', 13757),
 ('como', 12967),
 ('más', 12407),
 ('para', 11752),
 ('al', 11599),
 ('película', 11228),
 ('pero', 9414),
 ('o', 7125),
 ('sus', 5773),
 ('me', 5767),
 ('todo', 5631),
 ('sin', 5478),
 ('si', 5202),
 ('esta', 5065),
 ('ha', 4991),
 ('cine', 4987),
 ('le', 4804),
 ('muy', 4719),
 ('ya', 4567),
 ('nos', 4420),
 ('historia', 4289),
 ('este', 4087),
 ('ser', 4030),
 ('son', 3677),
 ('tiene', 3320),
 ('está', 3291),
 ('bien', 3251),
 ('hay', 3141),
 ('sobre', 3135),
 ('tan', 3095),
 ('aunque', 3019),
 ('uno', 2888),
 ('cuando', 2839),
 ('ni', 2835),
 ('hace', 2787),
 ('dos', 2668),
 ('ver', 2637),
 ('entre', 2632),
 ('algo', 2624),
 ('personajes', 2577),
 ('vez', 2539),
 ('poco', 2510

In [15]:
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

In [26]:
def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

In [25]:
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

In [24]:
def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

In [16]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

In [21]:
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [29]:
len(edits1('pelicul'))

390

In [30]:
len(list(edits2('pelicul')))

162150

In [32]:
correction('pelicul')

'pelicula'

In [ ]:
list(edits2('pelicul'))

In [33]:
wrong_words = [
    'pelicul',
    'guerr',
    'perona',
    'malha',
    'bueena',
    'rwsa',
    'entrida',
    'cime',
    'pelicula'
]

In [34]:
print('{0:20}{1:20}'.format('Palabra', 'Correción'))

for word in wrong_words:
    print('{0:20}{1:20}'.format(word, correction(word)))

Palabra             Correción           
pelicul             pelicula            
guerr               guerra              
perona              persona             
malha               mala                
bueena              buena               
rwsa                risa                
entrida             entrada             
cime                cine                
pelicula            pelicula            
